## 1. Carrega as bases de dados

In [ ]:
#!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org transformers
#!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org torch
#!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org tensorflow

In [1]:
import pandas as pd

###############################################################################
# Modelos
#MODELO = 'rufimelo/Legal-BERTimbau-sts-large-ma-v3'
#MAX_SEQ_LENGTH = 512

#MODELO = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
#MAX_SEQ_LENGTH = 128

#MODELO = 'neuralmind/bert-large-portuguese-cased'
#MAX_SEQ_LENGTH = 512

#MODELO = 'stjiris/bert-large-portuguese-cased-legal-mlm-sts-v1.0'
#MAX_SEQ_LENGTH = 512

#MODELO = 'stjiris/bert-large-portuguese-cased-legal-mlm-nli-sts-v1'
#MAX_SEQ_LENGTH = 512

#MODELO = 'Luciano/bert-base-portuguese-cased-finetuned-tcu-acordaos'
#MAX_SEQ_LENGTH = 512

MODELO = 'neuralmind/bert-base-portuguese-cased'
MAX_SEQ_LENGTH = 512

###############################################################################

CAMINHO_MODELO = MODELO.split("/")[-1]

# Pasta com os dados
PASTA_DADOS = './dados/'

# Pasta com os dados de jurisprudência já tratados
PASTA_JURIS_TCU = f'{PASTA_DADOS}outputs/1_tratamento_juris_tcu/'

# Pasta onde serão armazenados os resultados desse caderno
PASTA_RESULTADO_CADERNO = f'{PASTA_DADOS}outputs/6_gera_embeddings_js/{CAMINHO_MODELO}/'

# Tamanho do lote
#QUANTIDADE_DE_ITENS_CARREGADOS = 5

# Tamanho do lote
SOBRESCREVER_EMBEDDINGS = False

# Tamanho do lote
TAMANHO_DO_LOTE = 512

# Função que carrega os arquivos 
def carrega_juris_tcu():
    doc1 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_1.csv', sep='|')
    doc2 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_2.csv', sep='|')
    doc3 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_3.csv', sep='|')
    doc4 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_4.csv', sep='|')
    doc = pd.concat([doc1, doc2, doc3, doc4], ignore_index=True)

    return doc

In [3]:
from formatador import remove_html

# Carrega os arquivos para doc
doc = carrega_juris_tcu()
#doc = doc.head(QUANTIDADE_DE_ITENS_CARREGADOS)

#Remove tags do enunciado
doc['ENUNCIADO'] = doc['ENUNCIADO'].apply(remove_html)

#Transforma dataframe em dicionário
doc = doc.to_dict(orient='list')
print(doc.keys())

dict_keys(['KEY', 'NUMACORDAO', 'ANOACORDAO', 'COLEGIADO', 'AREA', 'TEMA', 'SUBTEMA', 'ENUNCIADO', 'EXCERTO', 'NUMSUMULA', 'DATASESSAOFORMATADA', 'AUTORTESE', 'FUNCAOAUTORTESE', 'TIPOPROCESSO', 'TIPORECURSO', 'INDEXACAO', 'INDEXADORESCONSOLIDADOS', 'PARAGRAFOLC', 'REFERENCIALEGAL', 'PUBLICACAOAPRESENTACAO', 'PARADIGMATICO'])


## 2. Tokenização do texto.

In [5]:
from transformers import AutoTokenizer

#Carregando o nokenizador Legal BERTimbau V3
model_ckpt = MODELO

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

C:\Users\santosr\AppData\Local\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\santosr\.cache\huggingface\hub\models--neuralmind--bert-base-portuguese-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [7]:
#Definição da função que realizará a tokenização em lotes
def tokenize(batch):
    return tokenizer(batch["ENUNCIADO"], padding=True, truncation=True, return_tensors='pt', max_length=MAX_SEQ_LENGTH)

In [9]:
# Aplicando a função de tokenização aos dados
doc_encoded = doc.copy()
tokenized_outputs = tokenize(doc_encoded)
tokenized_outputs

{'input_ids': tensor([[  101,   200, 22371,  ...,     0,     0,     0],
        [  101,   200, 22371,  ...,     0,     0,     0],
        [  101,   200, 22371,  ...,     0,     0,     0],
        ...,
        [  101,   200, 22371,  ...,     0,     0,     0],
        [  101,   200, 22371,  ...,     0,     0,     0],
        [  101,   200, 22371,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [21]:
tokenized_outputs.input_ids.shape

torch.Size([16045, 512])

In [23]:
# Armazenando input_ids, attention_mask e token_type_ids em doc_encoded
doc_encoded['input_ids'] = tokenized_outputs['input_ids']
doc_encoded['attention_mask'] = tokenized_outputs['attention_mask']
doc_encoded['token_type_ids'] = tokenized_outputs['token_type_ids']
doc_encoded.keys()

dict_keys(['KEY', 'NUMACORDAO', 'ANOACORDAO', 'COLEGIADO', 'AREA', 'TEMA', 'SUBTEMA', 'ENUNCIADO', 'EXCERTO', 'NUMSUMULA', 'DATASESSAOFORMATADA', 'AUTORTESE', 'FUNCAOAUTORTESE', 'TIPOPROCESSO', 'TIPORECURSO', 'INDEXACAO', 'INDEXADORESCONSOLIDADOS', 'PARAGRAFOLC', 'REFERENCIALEGAL', 'PUBLICACAOAPRESENTACAO', 'PARADIGMATICO', 'input_ids', 'attention_mask', 'token_type_ids'])

In [25]:
# Verifica se a tokenização foi realizada adequadamente
from itertools import islice

for enunciado, input_id in islice(zip(doc_encoded["ENUNCIADO"], doc_encoded["input_ids"]), 3):
    print(f"Enunciado: {enunciado}, \nInput IDs: {tokenizer.convert_ids_to_tokens(input_id)}\n")

Enunciado: SÚMULA TCU 43 (REVOGADA) : As pensões deferidas antes de 21/10/69, aos dependentes do pessoal, reformado, ou em atividade, da Polícia Militar e do Corpo de Bombeiros, transferido para o Estado da Guanabara, devem ser custeadas pela União, cabendo, porém, ao referido Estado a responsabilidade integral do pagamento decorrente dos reajustamentos posteriores., 
Input IDs: ['[CLS]', 'S', '##Ú', '##M', '##UL', '##A', 'T', '##C', '##U', '43', '(', 'R', '##E', '##V', '##O', '##GA', '##DA', ')', ':', 'As', 'pens', '##ões', 'defe', '##rida', '##s', 'antes', 'de', '21', '/', '10', '/', '69', ',', 'aos', 'dependentes', 'do', 'pessoal', ',', 'reforma', '##do', ',', 'ou', 'em', 'atividade', ',', 'da', 'Polícia', 'Militar', 'e', 'do', 'Corpo', 'de', 'Bombeiros', ',', 'transferido', 'para', 'o', 'Estado', 'da', 'Guanabara', ',', 'devem', 'ser', 'cus', '##te', '##adas', 'pela', 'União', ',', 'cabe', '##ndo', ',', 'porém', ',', 'ao', 'referido', 'Estado', 'a', 'responsa', '##bilidade', 'integ

## 3. Obtenção dos embeddings.

In [27]:
import torch
from transformers import AutoModel

# Carrega modelo

#Caso exista GPU utilize-a, caso contrário use a CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [29]:
# Função para agregaçção da última camada oculta pela média
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [31]:
# Função para extração da última camada oculta
def extract_hidden_states(batch):
    # Place model inputs on the GPU
    inputs = {k:v.to(device) for k,v in batch.items() 
              if k in tokenizer.model_input_names}
    # Extract last hidden states
    with torch.no_grad():
        model_output = model(**inputs)
    
    return batch['KEY'], model_output.last_hidden_state[:,0].cpu().numpy(), mean_pooling(model_output, inputs['attention_mask']).cpu().numpy()

In [33]:
# Função para dividir dicionário em lotes
def dividir_dicionario_em_lotes(dicionario, tamanho_do_lote):
    vetor_de_dicionarios = []
    max_len = max(len(v) for v in dicionario.values())  # Encontrando o vetor de valores mais longo
    
    for i in range(0, max_len, tamanho_do_lote):
        novo_dicionario = {chave: valores[i:i + tamanho_do_lote] for chave, valores in dicionario.items()}
        vetor_de_dicionarios.append(novo_dicionario)
    
    return vetor_de_dicionarios

In [35]:
import numpy as np

# Função para reconstruir dicionário a partir de lotes
def reconstruir_dicionario_a_partir_de_lotes(vetor_de_dicionarios):
    dicionario_reconstruido = {}
    
    # Inicializando listas vazias para cada chave no primeiro dicionário do vetor
    for chave in vetor_de_dicionarios[0].keys():
        dicionario_reconstruido[chave] = []
    
    # Iterando sobre cada dicionário no vetor e concatenando os valores para cada chave
    for dicionario_lote in vetor_de_dicionarios:
        for chave, valores in dicionario_lote.items():
            dicionario_reconstruido[chave].extend(valores)
    
    # Transforma numpy array em tensor
    dicionario_reconstruido['cls_hidden_state'] = torch.tensor(np.array(dicionario_reconstruido['cls_hidden_state']))
    dicionario_reconstruido['mean_hidden_state'] = torch.tensor(np.array(dicionario_reconstruido['mean_hidden_state']))
    
    return dicionario_reconstruido

In [37]:
CAMINHO_MODELO

'bert-base-portuguese-cased'

In [39]:
# Processar e salvar embeddings
import pickle
from tqdm import tqdm
import os

# Divide doc_encoded em lotes
doc_encoded_em_lotes = dividir_dicionario_em_lotes(doc_encoded, TAMANHO_DO_LOTE)

# Processa e salva embeddings
for i, dicionario in enumerate(tqdm(doc_encoded_em_lotes), start=1):
    
    caminho_arquivo = f'{PASTA_RESULTADO_CADERNO}{CAMINHO_MODELO}_embeddings_js_{i}.pickle'
    if  not SOBRESCREVER_EMBEDDINGS and os.path.exists(caminho_arquivo):
        continue

    key, cls_hidden_state, mean_hidden_state = extract_hidden_states(dicionario)
    
    # Cria estrutura que será salva em arquivo
    embeddings_js = {
        'key': key,
        'cls_hidden_state': cls_hidden_state,
        'mean_hidden_state': mean_hidden_state
    }
    
    # Gravando lote em um arquivo .pickle
    with open(caminho_arquivo, 'wb') as arquivo_pickle:
        pickle.dump(embeddings_js, arquivo_pickle)

100%|██████████| 32/32 [4:12:57<00:00, 474.28s/it]   


In [41]:
dicionario['input_ids'].size()
#dicionario['attention_mask']
#dicionario['token_type_ids']

torch.Size([173, 512])

In [43]:
doc_encoded_em_lotes[0]['KEY']

['JURISPRUDENCIA-SELECIONADA-85434',
 'JURISPRUDENCIA-SELECIONADA-85447',
 'JURISPRUDENCIA-SELECIONADA-85445',
 'JURISPRUDENCIA-SELECIONADA-85437',
 'JURISPRUDENCIA-SELECIONADA-85444',
 'JURISPRUDENCIA-SELECIONADA-85442',
 'JURISPRUDENCIA-SELECIONADA-85436',
 'JURISPRUDENCIA-SELECIONADA-85438',
 'JURISPRUDENCIA-SELECIONADA-85414',
 'JURISPRUDENCIA-SELECIONADA-85439',
 'JURISPRUDENCIA-SELECIONADA-85440',
 'JURISPRUDENCIA-SELECIONADA-85448',
 'JURISPRUDENCIA-SELECIONADA-85435',
 'JURISPRUDENCIA-SELECIONADA-85443',
 'JURISPRUDENCIA-SELECIONADA-85446',
 'JURISPRUDENCIA-SELECIONADA-85441',
 'JURISPRUDENCIA-SELECIONADA-84720',
 'JURISPRUDENCIA-SELECIONADA-1689',
 'JURISPRUDENCIA-SELECIONADA-16986',
 'JURISPRUDENCIA-SELECIONADA-13623',
 'JURISPRUDENCIA-SELECIONADA-15724',
 'JURISPRUDENCIA-SELECIONADA-15209',
 'JURISPRUDENCIA-SELECIONADA-14852',
 'JURISPRUDENCIA-SELECIONADA-11614',
 'JURISPRUDENCIA-SELECIONADA-22124',
 'JURISPRUDENCIA-SELECIONADA-28921',
 'JURISPRUDENCIA-SELECIONADA-30269',
 '

## 4. Cálculo da distância entre os embeddings.

In [45]:
# Função para restaurar embeddings dos arquivos pickle

import os
import pickle

def restaurar_doc_encoded_de_pickle(pasta_resultado_caderno):
    # Lista para armazenar os dicionários lidos dos arquivos .pickle
    doc_encoded_restaurado = []

    # Listando todos os arquivos .pickle no diretório especificado
    arquivos_pickle = [arq for arq in os.listdir(pasta_resultado_caderno) if arq.endswith('.pickle')]

    # Ordenando os arquivos pelo número (assumindo que os nomes dos arquivos seguem o padrão embeddings_js_X.pickle)
    arquivos_pickle.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))

    # Lendo cada arquivo .pickle e restaurando o dicionário
    for nome_arquivo in arquivos_pickle:
        caminho_arquivo = os.path.join(pasta_resultado_caderno, nome_arquivo)
        with open(caminho_arquivo, 'rb') as arquivo_pickle:
            dicionario_restaurado = pickle.load(arquivo_pickle)
            doc_encoded_restaurado.append(dicionario_restaurado)

    return reconstruir_dicionario_a_partir_de_lotes(doc_encoded_restaurado)

In [47]:
doc_encoded_restaurado = restaurar_doc_encoded_de_pickle(PASTA_RESULTADO_CADERNO)

In [49]:
import numpy as np

doc_hidden = doc_encoded.copy()
doc_hidden['cls_hidden_state'] = doc_encoded_restaurado['cls_hidden_state']
doc_hidden['mean_hidden_state'] = doc_encoded_restaurado['mean_hidden_state']
print(f"cls_hidden_state: {doc_hidden['cls_hidden_state'].size()}\n")
print(f"mean_hidden_state: {doc_hidden['mean_hidden_state'].size()}")

cls_hidden_state: torch.Size([16045, 768])

mean_hidden_state: torch.Size([16045, 768])


In [51]:
# Extraindo os embeddings de duas frases
embedding1_tensor = doc_hidden['mean_hidden_state'][0]
embedding2_tensor = doc_hidden['mean_hidden_state'][4]

# Normalizando os embeddings
embedding1_norm = embedding1_tensor / embedding1_tensor.norm()
embedding2_norm = embedding2_tensor / embedding2_tensor.norm()

# Calculando a similaridade por cosseno
cosine_similarity = torch.dot(embedding1_norm, embedding2_norm)

print(f"Similaridade por cosseno: {cosine_similarity.item()}")

Similaridade por cosseno: 0.8648619651794434


In [53]:
doc_encoded_restaurado

{'key': ['JURISPRUDENCIA-SELECIONADA-85434',
  'JURISPRUDENCIA-SELECIONADA-85447',
  'JURISPRUDENCIA-SELECIONADA-85445',
  'JURISPRUDENCIA-SELECIONADA-85437',
  'JURISPRUDENCIA-SELECIONADA-85444',
  'JURISPRUDENCIA-SELECIONADA-85442',
  'JURISPRUDENCIA-SELECIONADA-85436',
  'JURISPRUDENCIA-SELECIONADA-85438',
  'JURISPRUDENCIA-SELECIONADA-85414',
  'JURISPRUDENCIA-SELECIONADA-85439',
  'JURISPRUDENCIA-SELECIONADA-85440',
  'JURISPRUDENCIA-SELECIONADA-85448',
  'JURISPRUDENCIA-SELECIONADA-85435',
  'JURISPRUDENCIA-SELECIONADA-85443',
  'JURISPRUDENCIA-SELECIONADA-85446',
  'JURISPRUDENCIA-SELECIONADA-85441',
  'JURISPRUDENCIA-SELECIONADA-84720',
  'JURISPRUDENCIA-SELECIONADA-1689',
  'JURISPRUDENCIA-SELECIONADA-16986',
  'JURISPRUDENCIA-SELECIONADA-13623',
  'JURISPRUDENCIA-SELECIONADA-15724',
  'JURISPRUDENCIA-SELECIONADA-15209',
  'JURISPRUDENCIA-SELECIONADA-14852',
  'JURISPRUDENCIA-SELECIONADA-11614',
  'JURISPRUDENCIA-SELECIONADA-22124',
  'JURISPRUDENCIA-SELECIONADA-28921',
  'JUR

In [55]:
print(doc_hidden['ENUNCIADO'][0])
print(doc_hidden['ENUNCIADO'][1])
print(doc_hidden['ENUNCIADO'][4])

SÚMULA TCU 43 (REVOGADA) : As pensões deferidas antes de 21/10/69, aos dependentes do pessoal, reformado, ou em atividade, da Polícia Militar e do Corpo de Bombeiros, transferido para o Estado da Guanabara, devem ser custeadas pela União, cabendo, porém, ao referido Estado a responsabilidade integral do pagamento decorrente dos reajustamentos posteriores.
SÚMULA TCU 99 (REVOGADA) : Não pode ser imputado à conta dos Fundos de Participação (Constituição, art. 25) o percentual compulsório que incide sobre as receitas correntes próprias dos Estados, Municípios, Distrito Federal e Territórios, para a constituição do Programa de Formação do Patrimônio do Servidor Público (PASEP) .
SÚMULA TCU 63 (REVOGADA) : É lícita a vinculação de quotas dos Fundos de Participação, em garantia de contrato de abertura de crédito, financiamento, ou empréstimo celebrado pelos Estados, Distrito Federal ou Municípios, desde que observadas as normas emanadas do Poder Executivo Federal, notadamente a audiência pré